In [7]:
import tensorflow as tf
import numpy as np
import argparse
import socket
import time
import os
import scipy.misc
import sys
import glob
import umap
import matplotlib.pyplot as plt
from sklearn import datasets, decomposition, manifold, preprocessing


import os,sys,inspect
currentdir = "/usr/local/featureExtractionParty/external/pointnet_spine_ae"
sys.path.insert(0,currentdir) 
import provider
import importlib

import pandas as pd
from sklearn.cluster import KMeans

In [14]:
BATCH_SIZE = 1
NUM_POINT = 2048
MODEL_PATH = '/usr/local/featureExtractionParty/external/pointnet_spine_ae/log_model_manually_selected_trainingset_40_aligned/best_model_epoch_009.ckpt'
GPU_INDEX = 0
MODEL = importlib.import_module('models.model') # import network module
DUMP_DIR = 'dump'
if not os.path.exists(DUMP_DIR): os.mkdir(DUMP_DIR)
LOG_FOUT = open(os.path.join(DUMP_DIR, 'log_evaluate.txt'), 'w')
HOSTNAME = socket.gethostname()
print('test')

test


In [15]:
def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)
    

def evaluate(FILES, num_votes):
    is_training = False
     
    with tf.device('/gpu:'+str(GPU_INDEX)):
        
        pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
        print(pointclouds_pl.shape)
        is_training_pl = tf.placeholder(tf.bool, shape=())
        labels_pl_rep = tf.placeholder(tf.float32,shape=(1))

        # simple model
        pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl)
        print("Size of embedding")
        print(pred.shape)
        
        # Add ops to save and restore all the variables.
        saver = tf.train.Saver()
        
    # Create a session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = True
    sess = tf.Session(config=config)

    # Restore variables from disk.
    saver.restore(sess, MODEL_PATH)
    log_string("Model restored.")

    ops = {'pointclouds_pl': pointclouds_pl,
           'labels_pl': labels_pl_rep,
           'is_training_pl': is_training_pl,
           'pred': pred,
           'embedding': end_points['embedding']
           }
    eval_one_epoch(FILES, sess, ops, num_votes)
    #features,files = eval_one_epoch(FILES, sess, ops, num_votes)
    #features = np.array(features)
    #return features,files
    
def eval_one_epoch(FILES, sess, ops, num_votes=1, topk=1):
    features = []
    filenames = []
    
    is_training = False
    fout = open(os.path.join(DUMP_DIR, 'pred_label.txt'), 'w')
    for fn in range(len(FILES)):
        outfile = FILES[fn].replace(".off","_ae_model_manualV3.txt")
        if not os.path.exists(outfile):
        #if 1 ==1:
            log_string('----'+str(fn)+'----')
            print(FILES[fn])
            current_data, current_label = provider.loadAllOffDataFile(FILES[fn], NUM_POINT)
            
            if current_data is not None:
                feed_dict = {ops['pointclouds_pl']: current_data,
                                     ops['labels_pl']: current_label,
                                     ops['is_training_pl']: is_training}
                pred_val = sess.run([ops['embedding']], feed_dict=feed_dict)
                #pred_val = sess.run([ops['pred']], feed_dict=feed_dict)

                np.savetxt(outfile,np.squeeze(pred_val))
                
                print (outfile)
                #features.append(np.squeeze(pred_val))
    
def loadfeatures(feat_files):
    features = []
    #synapse_distances = []
    index = 0
    for f in feat_files:
        index += 1
        v = np.loadtxt(f)
        features.append(v)
        #curdistfilename = f.replace('ae_model_v2.txt','distance.txt')
        #synapse_distances.append(np.loadtxt(curdistfilename))
    return features

def dist2pt (features, pt):
    features = np.array(features)
    pt = np.array(pt)
    dists = np.linalg.norm(features-pt[np.newaxis,:],axis=1) 
    return np.argmin(dists,axis=0)